In [817]:
import pandas as pd
import pywt
import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from torch.utils.data import DataLoader

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#torch.backends.cudnn.benchmark = True

## Wavelet transform

In [818]:
def van_haar(data):
    ncol = data.shape[1]
    nrow = data.shape[0]
    for i in range(ncol):
        cur_col = data[:,i].copy()
        (cA, cD) = pywt.dwt(cur_col, 'haar')
        new_col = np.reshape(np.concatenate((cA,cD), 0),(nrow,1))
        data = np.hstack((data,new_col))
    data = data.reshape(nrow,-1)
    return data

#van_haar(test.to_numpy()).shape

## PyTorch

In [819]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(
            self, 
            n_inputs, 
            n_outputs, 
            kernel_size, 
            stride,
            dilation,
            padding,
            dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(
            nn.Conv1d(
                n_inputs, 
                n_outputs, 
                kernel_size,
                stride=stride, 
                padding=padding,
                dilation=dilation))
        self.pad = Chomp1d(padding)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
        self.conv2 = weight_norm(
            nn.Conv1d(
                n_outputs, 
                n_outputs, 
                kernel_size,
                stride=stride, 
                padding=padding, 
                dilation=dilation))
        self.net = nn.Sequential(
            self.pad, 
            self.conv1, 
            self.relu, 
            self.dropout,
            self.pad, 
            self.conv2, 
            self.relu, 
            self.dropout)
        
        self.downsample = nn.Conv1d(
            n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, 
                                     out_channels, 
                                     kernel_size, 
                                     stride=1, 
                                     dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, 
                                     dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [820]:
def reference_transform(tensor):
    array = tensor.numpy()
    out1, out2= pywt.dwt(array, "haar")
    out1 = torch.from_numpy(out1)
    out2 = torch.from_numpy(out2)
    
    #concatenate each channel to be able to concatenate it to the untransformed data
    #everything will then be split when fed to the network
    return torch.cat((out1, out2),-1)
    

In [821]:
class WaveletPart(nn.Module):

    def __init__(self, input_size, output_size):
        super(WaveletPart, self).__init__()

        # used two different layers here as in the paper but in the github code, they are the same
        self.fc1 = nn.Linear(input_size, output_size)
        self.fc2 = nn.Linear(input_size, output_size)

        self.input_size = input_size

        self.haar = reference_transform

    def init_weight(self):
        self.fc1.weight.data.normal_(0, 0.01)
        self.fc1.bias.data.normal_(0, 0.01)
        self.fc2.weight.data.normal_(0, 0.01)
        self.fc2.bias.data.normal_(0, 0.01)

    def forward(self, x):
        # split the wavelet transformed data along third dim
        x1, x2 = torch.split(x, 500, 2)

        # reshape everything to feed to the linear layer
        bsize = x.size()[0]
        x1 = self.fc1(x1.reshape((bsize, -1, 1)).squeeze())
        x2 = self.fc2(x2.reshape((bsize, -1, 1)).squeeze())
        x1 = x1.reshape(bsize, -1)
        x2 = x2.reshape(bsize, -1)
        return torch.cat((x1,x2),-1)

In [822]:
class Driver2Vec(nn.Module):
    def __init__(
            self, 
            input_size, 
            input_length, 
            num_channels,
            output_size, 
            kernel_size, 
            dropout,
            fc_output_size = 15):
        super(Driver2Vec, self).__init__()
        
        self.tcn = TemporalConvNet(input_size, 
                                   num_channels, 
                                   kernel_size=kernel_size, 
                                   dropout=dropout)
        
        self.haar = WaveletPart(input_size*input_length//2, fc_output_size)

        linear_size = num_channels[-1] + fc_output_size*2
        self.input_length = input_length

        self.input_bn = nn.BatchNorm1d(linear_size)
        self.linear = nn.Linear(linear_size, output_size)

    def forward(self, inputs):
        """Inputs have to have dimension (N, C_in, L_in*2)
        the base time series, and the two wavelet transform channel are concatenated along dim2"""
        
        # split the inputs, in the last dim, first is the unchanged data, then 
        # the wavelet transformed data
        input_tcn, input_haar = torch.split(inputs, self.input_length, 2)

        # feed each one to their corresponding network
        y1 = self.tcn(input_tcn)
        y1 = y1[:,:,-1] # for the TCN, only the last output element interests us
        y2 = self.haar(input_haar)

        concat = torch.cat((y1,y2),1)
        bsize = concat.shape[0]
        if bsize > 1: # issue when the batch size is 1, can't batch normalize it
            out = self.input_bn(concat)
        else:
            out = concat
        out = self.linear(out)
        
        return out

In [823]:
input_channels = 31
channel_sizes = [25,25,32]
output_size = 62
kernel_size = 16
dropout = 0.1
model = Driver2Vec(input_channels, 1000, channel_sizes, output_size, kernel_size=kernel_size, dropout=dropout)
model.to(device)

Driver2Vec(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(31, 25, kernel_size=(16,), stride=(1,), padding=(15,))
        (pad): Chomp1d()
        (relu): ReLU()
        (dropout): Dropout(p=0.1, inplace=False)
        (conv2): Conv1d(25, 25, kernel_size=(16,), stride=(1,), padding=(15,))
        (net): Sequential(
          (0): Chomp1d()
          (1): Conv1d(31, 25, kernel_size=(16,), stride=(1,), padding=(15,))
          (2): ReLU()
          (3): Dropout(p=0.1, inplace=False)
          (4): Chomp1d()
          (5): Conv1d(25, 25, kernel_size=(16,), stride=(1,), padding=(15,))
          (6): ReLU()
          (7): Dropout(p=0.1, inplace=False)
        )
        (downsample): Conv1d(31, 25, kernel_size=(1,), stride=(1,))
      )
      (1): TemporalBlock(
        (conv1): Conv1d(25, 25, kernel_size=(16,), stride=(1,), padding=(30,), dilation=(2,))
        (pad): Chomp1d()
        (relu): ReLU()
        (dropout): Dropout(p=0.1, i

In [824]:
import os

def preprocess(df):
    return (
        df.drop(
            ["FOG",
             "FOG_LIGHTS",
             "FRONT_WIPERS",
             "HEAD_LIGHTS",
             "RAIN",
             "REAR_WIPERS",
             "SNOW",
            ], axis=1
        )
    )

def load_dataset(input_dir):
    x=[]
    y=[]
    for dir,_,files in os.walk(input_dir):
        for file in files:
            label = int(file.split("_")[1])
            df = pd.read_csv(dir + "/" + file, index_col=0)
            df = preprocess(df).to_numpy().transpose()
            x.append(torch.from_numpy(df).float())
            y.append(label)
    return x,y
        
class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, input_dir, train=True):
        'Initialization'
        self.data, self.labels = load_dataset(input_dir)
        self.index = [i for i in range(len(self.data))]
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        X_anchor = self.data[index]
        anchor_wvlt = reference_transform(X_anchor)
        y_anchor = self.labels[index]

        if self.train:
            # create list of possible positive index samples
            positive_list = [
                i for i in self.index if self.labels[i] == y_anchor and i != index]
            positive_index = np.random.choice(positive_list)
            positive = self.data[positive_index]
            # get the wavelet transform of that sample (the two channels are concatenated along the last dim)
            positive_wvlt = reference_transform(positive)
            
            # same here
            negative_list = [
                i for i in self.index if self.labels[i] != y_anchor]
            negative_index = np.random.choice(negative_list)
            negative = self.data[negative_index]
            negative_wvlt = reference_transform(negative)

            # concatenate the data for the TCN and the haar wavelet transform
            # they will be split in the forward pass
            return torch.cat((X_anchor, anchor_wvlt),1), \
                torch.cat((positive, positive_wvlt),1), \
                torch.cat((negative, negative_wvlt),1), \
                y_anchor

        return X_anchor, y_anchor



In [825]:

# datasets parameters
params = {'batch_size': 3,
          'shuffle': False,
          'num_workers': 1}

training_set = Dataset("./dataset")
training_generator = DataLoader(training_set, **params)

loss = nn.TripletMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [826]:
for anchor, positive, negative, label in training_generator:
    anchor = anchor.to(device)
    positive = positive.to(device)
    negative = negative.to(device)

    optimizer.zero_grad()
    y_anchor = model(anchor)
    y_positive = model(positive)
    y_negative = model(negative)

